# download spicy products for ASO rasters

This notebook contains functions to run spicy-snow for the region/date of each snow depth raster in a directory.

NOTES:
- Changed sentinel1.py in spicy to resample to 60 m pixel spacing rather than default 90.
- Changed user_dates.py to have start date on September 1 instead of August 1
- Changed sentinel1.py to download dem in addition to other hyp3 outputs

In [1]:
# # install spicy-snow in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install spicy-snow

In [2]:
# import spicy-snow functions and other required packages
# depending on your environment, you may need to install some of these
from spicy_snow.retrieval import retrieve_snow_depth
from spicy_snow.IO.user_dates import get_input_dates
import geopandas as gpd
from pathlib import Path
from shapely import geometry
from glob import glob
import rasterio as rio
import os

In [3]:
# create groups by year, utm zone, and location
utm10n_2016a = ['ASO_50M_SD_USWAOL_20160329_clean', 'ASO_50M_SD_USWAOL_20160208_clean']
utm10n_2022a = ['ASO_50M_SD_Truckee_20220514_clean', 'ASO_50M_SD_Feather_20220429_clean', 'ASO_50M_SD_Feather_20220310_clean', 
                'ASO_50M_SD_Feather_20220331_clean', 'ASO_50M_SD_Truckee_20220310_clean', 'ASO_50M_SD_Truckee_20220408_clean']
utm10n_2023a = ['ASO_50M_SD_American_20230413_clean', 'ASO_50M_SD_American_20230428_clean', 'ASO_50M_SD_American_20230131_clean', 
                'ASO_50M_SD_American_20230602_clean', 'ASO_50M_SD_American_20230602_clean', 'ASO_50M_SD_American_20230602_clean', 
                'ASO_50M_SD_Feather_20230206_clean', 'ASO_50M_SD_Feather_20230618_clean', 'ASO_50M_SD_LowerPit_20230425_clean', 
                'ASO_50M_SD_LowerPit_20230212_clean', 'ASO_50M_SD_LowerPit_20230511_clean', 'ASO_50M_SD_SacramentoMcCloud_20230209_clean', 
                'ASO_50M_SD_SacramentoMcCloud_20230515_clean', 'ASO_50M_SD_Yuba_20230625_clean', 'ASO_50M_SD_Yuba_20230531_clean', 
                'ASO_50M_SD_Yuba_20230128_clean', 'ASO_50M_SD_Yuba_20230426_clean', 'ASO_50M_SD_Yuba_20230405_clean', 
                'ASO_50M_SD_Truckee_20230511_clean', 'ASO_50M_SD_Truckee_20230608_clean', 'ASO_50M_SD_Truckee_20230409_clean']
utm10n_2023b = ['ASO_50M_SD_Sprague_20230514_clean']

utm11n_2016a = ['ASO_50M_SD_USCALB_20160626_clean', 'ASO_50M_SD_USCALB_20160621_clean', 'ASO_50M_SD_USCALB_20160614_clean', 
                'ASO_50M_SD_USCALB_20160607_clean', 'ASO_50M_SD_USCALB_20160509_clean', 'ASO_50M_SD_USCARC_20160412_clean', 
                'ASO_50M_SD_USCATB_20160326_clean', 'ASO_50M_SD_USCATB_20160401_clean', 'ASO_50M_SD_USCATB_20160407_clean', 
                'ASO_50M_SD_USCATB_20160416_clean', 'ASO_50M_SD_USCATB_20160426_clean', 'ASO_50M_SD_USCATB_20160509_clean', 
                'ASO_50M_SD_USCATB_20160527_clean', 'ASO_50M_SD_USCATB_20160607_clean', 'ASO_50M_SD_USCATB_20160708_clean']
utm11n_2017a = ['ASO_50M_SD_USCALB_20170815_clean', 'ASO_50M_SD_USCALB_20170128_clean', 'ASO_50M_SD_USCALV_20170717_clean', 
                'ASO_50M_SD_USCARC_20170717_clean', 'ASO_50M_SD_USCASF_20170718_clean', 'ASO_50M_SD_USCASJ_20170129_clean', 
                'ASO_50M_SD_USCASJ_20170719_clean', 'ASO_50M_SD_USCASJ_20170815_clean', 'ASO_50M_SD_USCATB_20170717_clean', 
                'ASO_50M_SD_USCATB_20170727_clean', 'ASO_50M_SD_USCATB_20170816_clean']
utm11n_2018a = ['ASO_50M_SD_USCALB_20180601_clean', 'ASO_50M_SD_USCALB_20180422_clean', 'ASO_50M_SD_USCAJW_20180423_clean', 
                'ASO_50M_SD_USCAJW_20180602_clean', 'ASO_50M_SD_USCAKC_20180426_clean', 'ASO_50M_SD_USCAKN_20180426_clean', 
                'ASO_50M_SD_USCAMB_20180425_clean', 'ASO_50M_SD_USCASF_20180423_clean', 'ASO_50M_SD_USCASF_20180601_clean', 
                'ASO_50M_SD_USCASJ_20180422_clean', 'ASO_50M_SD_USCASJ_20180601_clean', 'ASO_50M_SD_USCATB_20180423_clean', 
                'ASO_50M_SD_USCATB_20180528_clean']
utm11n_2019a = ['ASO_50M_SD_USCALB_20190715_clean', 'ASO_50M_SD_USCALB_20190703_clean', 'ASO_50M_SD_USCALB_20190611_clean', 
                'ASO_50M_SD_USCALB_20190501_clean', 'ASO_50M_SD_USCALB_20190309_clean', 'ASO_50M_SD_USCAJW_20190605_clean', 
                'ASO_50M_SD_USCAJW_20190315_clean', 'ASO_50M_SD_USCAKC_20190316_clean', 'ASO_50M_SD_USCAKC_20190326_clean', 
                'ASO_50M_SD_USCAKC_20190418_clean', 'ASO_50M_SD_USCAKC_20190427_clean', 'ASO_50M_SD_USCAKC_20190428_clean', 
                'ASO_50M_SD_USCAKC_20190608_clean', 'ASO_50M_SD_USCAKC_20190611_clean', 'ASO_50M_SD_USCAKN_20190417_clean', 
                'ASO_50M_SD_USCAKW_20190317_clean', 'ASO_50M_SD_USCAKW_20190324_clean', 'ASO_50M_SD_USCAKW_20190421_clean', 
                'ASO_50M_SD_USCAMB_20190329_clean', 'ASO_50M_SD_USCAMB_20190604_clean', 'ASO_50M_SD_USCAMB_20190703_clean', 
                'ASO_50M_SD_USCAMB_20190716_clean', 'ASO_50M_SD_USCASF_20190317_clean', 'ASO_50M_SD_USCASF_20190502_clean', 
                'ASO_50M_SD_USCASF_20190609_clean', 'ASO_50M_SD_USCASF_20190704_clean', 'ASO_50M_SD_USCASF_20190714_clean', 
                'ASO_50M_SD_USCASJ_20190325_clean', 'ASO_50M_SD_USCASJ_20190501_clean', 'ASO_50M_SD_USCASJ_20190614_clean', 
                'ASO_50M_SD_USCASJ_20190704_clean', 'ASO_50M_SD_USCASJ_20190713_clean', 'ASO_50M_SD_USCATE_20190324_clean', 
                'ASO_50M_SD_USCATE_20190417_clean', 'ASO_50M_SD_USCATE_20190503_clean', 'ASO_50M_SD_USCATE_20190613_clean', 
                'ASO_50M_SD_USCATE_20190705_clean']
utm11n_2020a = ['ASO_50M_SD_Tuolumne_20200507_clean', 'ASO_50M_SD_Tuolumne_20200413_clean', 'ASO_50M_SD_SanJoaquin_20200523_clean', 
                'ASO_50M_SD_SanJoaquin_20200608_clean', 'ASO_50M_SD_Merced_20200521_clean', 'ASO_50M_SD_Merced_20200604_clean', 
                'ASO_50M_SD_Merced_20200507_clean', 'ASO_50M_SD_Merced_20200413_clean']
utm11n_2021a = ['ASO_50M_SD_Tuolumne_20210224_clean', 'ASO_50M_SD_Tuolumne_20210429_clean', 'ASO_50M_SD_SanJoaquin_20210503_clean', 
                'ASO_50M_SD_SanJoaquin_20210226_clean', 'ASO_50M_SD_SanJoaquin_20210331_clean', 'ASO_50M_SD_Merced_20210429_clean', 
                'ASO_50M_SD_Merced_20210526_clean', 'ASO_50M_SD_Kaweah_20210504_clean', 'ASO_50M_SD_Kaweah_20210423_clean']
utm11n_2022a = ['ASO_50M_SD_Carson_20220406_clean', 'ASO_50M_SD_Tuolumne_20220518_clean', 'ASO_50M_SD_Tuolumne_20220204_clean', 
                'ASO_50M_SD_Tuolumne_20220228_clean', 'ASO_50M_SD_Tuolumne_20220405_clean', 'ASO_50M_SD_Tuolumne_20220429_clean', 
                'ASO_50M_SD_SanJoaquin_20220316_clean', 'ASO_50M_SD_SanJoaquin_20220206_clean', 'ASO_50M_SD_SanJoaquin_20220430_clean', 
                'ASO_50M_SD_SanJoaquin_20220417_clean', 'ASO_50M_SD_Merced_20220513_clean', 'ASO_50M_SD_Merced_20220301_clean', 
                'ASO_50M_SD_Merced_20220428_clean', 'ASO_50M_SD_Merced_20220412_clean', 'ASO_50M_SD_Kaweah_20220517_clean', 
                'ASO_50M_SD_Kaweah_20220327_clean', 'ASO_50M_SD_Kaweah_20220308_clean', 'ASO_50M_SD_Kaweah_20220429_clean']
utm11n_2023a = ['ASO_50M_SD_Carson_20230607_clean', 'ASO_50M_SD_Carson_20230331_clean', 'ASO_50M_SD_Carson_20230510_clean', 
                'ASO_50M_SD_UpperOwensBasin_20230527_clean', 'ASO_50M_SD_UpperOwensBasin_20230615_clean', 'ASO_50M_SD_UpperOwensBasin_20230702_clean', 
                'ASO_50M_SD_Tuolumne_20230316_clean', 'ASO_50M_SD_Tuolumne_20230302_clean', 'ASO_50M_SD_Tuolumne_20230626_clean',
                'ASO_50M_SD_Tuolumne_20230601_clean', 'ASO_50M_SD_Tuolumne_20230124_clean', 'ASO_50M_SD_Tuolumne_20230427_clean',
                'ASO_50M_SD_SanJoaquin_20230523_clean', 'ASO_50M_SD_SanJoaquin_20230317_clean', 'ASO_50M_SD_SanJoaquin_20230623_clean', 
                'ASO_50M_SD_SanJoaquin_20230121_clean', 'ASO_50M_SD_SanJoaquin_20230414_clean', 'ASO_50M_SD_Mono_20230527_clean', 
                'ASO_50M_SD_Mono_20230615_clean', 'ASO_50M_SD_Mono_20230702_clean', 'ASO_50M_SD_Merced_20230515_clean', 
                'ASO_50M_SD_Merced_20230326_clean', 'ASO_50M_SD_Merced_20230303_clean', 'ASO_50M_SD_Merced_20230608_clean', 
                'ASO_50M_SD_Merced_20230131_clean', 'ASO_50M_SD_Kings_20230528_clean', 'ASO_50M_SD_Kings_20230331_clean', 
                'ASO_50M_SD_Kings_20230316_clean', 'ASO_50M_SD_Kings_20230630_clean', 'ASO_50M_SD_Kings_20230421_clean', 
                'ASO_50M_SD_Kings_20230122_clean', 'ASO_50M_SD_Kern_20230326_clean', 'ASO_50M_SD_Kern_20230603_clean', 
                'ASO_50M_SD_Kern_20230204_clean', 'ASO_50M_SD_Kern_20230429_clean', 'ASO_50M_SD_Kaweah_20230307_clean', 
                'ASO_50M_SD_Kaweah_20230602_clean', 'ASO_50M_SD_Kaweah_20230127_clean', 'ASO_50M_SD_Kaweah_20230423_clean',
                'ASO_50M_SD_Kaweah_20230405_clean']

utm12n_2021a = ['ASO_50M_SD_Dolores_20210514_clean', 'ASO_50M_SD_Dolores_20210420_clean']
utm12n_2022a = ['ASO_50M_SD_Dolores_20220510_clean', 'ASO_50M_SD_Dolores_20220415_clean']
utm12n_2022b = ['ASO_50M_SD_WindRiver_20220611_clean', 'ASO_50M_SD_GreenRiver_20220611_clean']
utm12n_2023a = ['ASO_50M_SD_Dolores_20230525_clean', 'ASO_50M_SD_Dolores_20230406_clean']

utm13n_2016a = ['ASO_50M_SD_USCOCJ_20160403_clean', 'ASO_50M_SD_USCORG_20160403_clean']
utm13n_2018a = ['ASO_50M_SD_USCOGE_20180331_clean', 'ASO_50M_SD_USCOGE_20180524_clean', 'ASO_50M_SD_USCOGT_20180330_clean', 
                'ASO_50M_SD_USCOCB_20180330_clean']
utm13n_2019a = ['ASO_50M_SD_BlueRiver_20190419_clean', 'ASO_50M_SD_BlueRiver_20190624_clean', 'ASO_50M_SD_TenMileCk_20190613_clean', 
                'ASO_50M_SD_USCOBR_20190419_clean', 'ASO_50M_SD_USCOBR_20190624_clean', 'ASO_50M_SD_USCOCM_20190407_clean', 
                'ASO_50M_SD_USCOCM_20190610_clean', 'ASO_50M_SD_USCOGE_20190407_clean', 'ASO_50M_SD_USCOGE_20190610_clean', 
                'ASO_50M_SD_USCOGT_20190408_clean', 'ASO_50M_SD_USCOGT_20190609_clean']
utm13n_2021a = ['ASO_50M_SD_Animas_20210419_clean', 'ASO_50M_SD_BlueRiver_20210418_clean', 'ASO_50M_SD_BlueRiver_20210524_clean', 
                'ASO_50M_SD_Conejos_20210420_clean', 'ASO_50M_SD_Conejos_20210516_clean']
utm13n_2022a = ['ASO_50M_SD_Blue_20220419_clean', 'ASO_50M_SD_Blue_20220526_clean', 'ASO_50M_SD_Conejos_20220510_clean', 
                'ASO_50M_SD_EastRiver_20220518_clean', 'ASO_50M_SD_Gunnison_20220421_clean', 'ASO_50M_SD_GunnisonTaylor_20220421_clean', 
                'ASO_50M_SD_GunnisonTaylor_20220525_clean', 'ASO_50M_SD_Conejos_20220415_clean', 'ASO_50M_SD_WindyGap_20220418_clean', 
                'ASO_50M_SD_WindyGap_20220526_clean']
utm13n_2023a = ['ASO_50M_SD_BigThompsonLittleThompson_20230521_clean', 'ASO_50M_SD_BlueRiver_20230416_clean', 'ASO_50M_SD_BlueRiver_20230529_clean', 
                'ASO_50M_SD_BoulderCreek_20230509_clean', 'ASO_50M_SD_ClearCreek_20230509_clean', 'ASO_50M_SD_Conejos_20230505_clean', 
                'ASO_50M_SD_EastRiver_20230401_clean', 'ASO_50M_SD_EastRiver_20230523_clean', 'ASO_50M_SD_Poudre_20230522_clean',
                'ASO_50M_SD_RoaringFork_20230411_clean', 'ASO_50M_SD_RoaringFork_20230528_clean', 'ASO_50M_SD_SouthPlatte_20230416_clean',
                'ASO_50M_SD_SouthPlatte_20230526_clean', 'ASO_50M_SD_StVrainLefthand_20230521_clean', 'ASO_50M_SD_Taylor_20230401_clean', 
                'ASO_50M_SD_TaylorAndLottis_20230523_clean', 'ASO_50M_SD_WindyGap_20230527_clean', 'ASO_50M_SD_WindyGap_20230416_clean']

In [4]:
def date_bbox_from_groups(group, dir_path):
    bbox = {'left':100000000, 'bottom':100000000, 'right':0, 'top':0}
    latest_date = 0
    for fn in group:
        path = f'{dir_path}/{fn}.tif'
        raster_bounds = rio.open(path).bounds

        if bbox['left'] > raster_bounds[0]:
            bbox['left'] = raster_bounds[0]
        if bbox['bottom'] > raster_bounds[1]:
            bbox['bottom'] = raster_bounds[1]
        if bbox['right'] < raster_bounds[2]:
            bbox['right'] = raster_bounds[2]
        if bbox['top'] < raster_bounds[3]:
            bbox['top'] = raster_bounds[3]

        date = int(fn.split('_')[-2])
        if date > latest_date:
            latest_date = date

    return str(latest_date), bbox

In [5]:
def spicy_for_aso_group(group, dir_path):
    '''
    Run spicy for the combined extent and latest date of group of ASO rasters
    '''
    # get latest date and combined bbox for group
    date, bbox = date_bbox_from_groups(group, dir_path)
    
    raster = rio.open(f'{dir_path}/{group[0]}.tif')
    gdf = gpd.GeoDataFrame({"id":1,"geometry":[geometry.box(*bbox.values())]}, crs=raster.crs)
    area = gdf.to_crs('4326').geometry.values[0]

    dates = get_input_dates(date)
    group_name = f'{group=}'.split('=')[0]
        
    # define output directory and file name
    out_nc = Path(f'..data/spicy/{group_name.split("_")[0]}/spicy_{group_name}_{dates[0]}_{dates[1]}.nc').expanduser()
        
    if os.path.exists(out_nc):
            print('spicy-snow output already exists, skipping')
            
    else:
        # run spicy-snow for ASO raster group
        spicy_ds = retrieve_snow_depth(area = area,
                                       dates = dates, 
                                       work_dir = Path('/mnt/Backups/gbrench/repos/crunchy-snow/data/tmp').expanduser(), 
                                       job_name = f'spicy_{group_name}_{dates[0]}_{dates[1]}',
                                       existing_job_name = f'spicy_{group_name}_{dates[0]}_{dates[1]}',
                                       debug=False,
                                       outfp=out_nc)
            
        #!rm /tmp/spicy/tmp/S1*.tif

In [ ]:
path = '../data/ASO/ASO_50m_SD_cleaned/utm13n'
spicy_for_aso_group(utm13n_2023a, path)

(spicy-snow spicy_snow.utils.spicy_logging INFO) Found 402 results


Submitting s1 jobs: 100%|███████████████████████████████████████████████████| 402/402 [03:24<00:00,  1.97it/s]

(spicy-snow spicy_snow.utils.spicy_logging INFO) Watching 402 jobs. This may take a while...


  0%|          | 0/402 [timeout in 10800 s]

In [ ]:
# def spicy_for_aso(dir_path):
#     '''
#     Run spicy for the extent and date of ASO rasters
#     '''
#     path_list = glob(f'{dir_path}/*ASO_50M_SD*.tif')
    
#     for i, path in enumerate(path_list):
#         print(f'-------\nworking on {path.split("/")[-1]}, {i+1}/{len(path_list)}\n-------')
#         # get bounding box from ASO raster
#         raster = rio.open(path)
#         gdf = gpd.GeoDataFrame({"id":1,"geometry":[geometry.box(*raster.bounds)]}, crs=raster.crs)
#         area = gdf.to_crs('4326').geometry.values[0]
        
#         # get acquistion date from ASO raster
#         dates = get_input_dates(path.split('/')[-1].split('_')[-2])
        
#         # define output directory and file name
#         site_name = path.split('/')[-1].split('_')[-3]
#         utm_zone = path.split('/')[-2]
#         out_nc = Path(f'..data/spicy/{utm_zone}/spicy_{site_name}_{dates[0]}_{dates[1]}.nc').expanduser()
        
#         if os.path.exists(out_nc):
#             print('spicy-snow output already exists, skipping')
            
#         else:
#             # run spicy-snow for ASO raster
#             spicy_ds = retrieve_snow_depth(area = area,
#                                            dates = dates, 
#                                            work_dir = Path('/mnt/Backups/gbrench/repos/crunchy-snow/data/tmp').expanduser(), 
#                                            job_name = f'spicy_{site_name}_{dates[0]}_{dates[1]}',
#                                            existing_job_name = f'spicy_{site_name}_{dates[0]}_{dates[1]}',
#                                            debug=False,
#                                            outfp=out_nc)
            
#             #!rm /tmp/spicy/tmp/S1*.tif

In [ ]:
# path = '../data/ASO/ASO_50m_SD_cleaned/utm12n'
# spicy_for_aso(path)